<a href="https://colab.research.google.com/github/jeffersonramelo/workshop-ufu/blob/main/base_copomEN_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
# Ler o arquivo XLSX
dados = pd.read_excel('/content/atascopomEN_ML.xlsx')

In [ ]:
#vizualizar dados
dados.head()

In [ ]:
#quantos dados positivo e negativos?
dados.sentimento.value_counts()

negativo    142
positivo     72
Name: sentimento, dtype: int64

In [ ]:
# Acessar os textos contidos nas celulas, só substituir o número da celula que deseja vizualizar
celula = dados.loc[0, 'texto']

# Imprimir o conteúdo da célula
print(celula)


th_x000d_ 254 minutes meeting the_x000d_ monetary policy committee copom_x000d_ may 23 2023_x000d_ _x000d_ _x000d_ _x000d_ _x000d_ 1 bcbgovbr_x000d_ _x000d_ 254th meeting_x000d_ copom_x000d_ may 23_x000d_ _x000d_ _x000d_ date may 23 2023_x000d_ _x000d_ place bcb headquarters meeting rooms 8th floor 52 53 morning_x000d_ 20th floor 53 afternoon brasilia df brazil_x000d_ _x000d_ starting ending may 2 1013 1221 pm 228 pm 638 pm_x000d_ times may 3 1006 1115 pm 234 pm 646 pm_x000d_ _x000d_ attendance_x000d_ copom members roberto de oliveira campos neto governor_x000d_ diogo abry guillen_x000d_ fernanda magalhães rumenos guardado_x000d_ maurício costa de moura_x000d_ otávio ribeiro damaso_x000d_ paulo sérgio neves de souza_x000d_ renato dias de brito gomes_x000d_ _x000d_ department heads alan da silva andrade mendes department foreign reserves_x000d_ charge technical andré minella research department also present afternoon 53_x000d_ presentations attending andré de oliveira amante open market

In [4]:
import re

# Função para remover caracteres especiais
def remove_caracteres_especiais(texto):
    texto = re.sub(r'[^\w\s]', '', texto)
    return texto

# Aplicar a função na coluna de texto
dados['texto'] = dados['texto'].apply(remove_caracteres_especiais)

In [5]:
# Transformar todas as letras maiúsculas em minúsculas na coluna de texto
dados['texto'] = dados['texto'].str.lower()

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Obter a lista de stop words em inglês
stop_words = set(stopwords.words('english'))

# Função para remover as stop words
def remove_stop_words(texto):
    tokens = word_tokenize(texto)  # Tokenização do texto
    tokens_sem_stop_words = [word for word in tokens if word.casefold() not in stop_words]  # Remoção das stop words
    texto_sem_stop_words = ' '.join(tokens_sem_stop_words)  # Reconstrução do texto sem stop words
    return texto_sem_stop_words

# Aplicar a função na coluna de texto
dados['texto'] = dados['texto'].apply(remove_stop_words)

In [9]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
from nltk.stem import PorterStemmer

# Função para realizar a stemização das palavras
def stem_words(texto):
    stemmer = PorterStemmer()
    tokens = word_tokenize(texto)  # Tokenização do texto
    stemmed_tokens = [stemmer.stem(word) for word in tokens]  # Stemização das palavras
    texto_stemizado = ' '.join(stemmed_tokens)  # Reconstrução do texto stemizado
    return texto_stemizado

# Aplicar a função na coluna de texto
dados['texto'] = dados['texto'].apply(stem_words)


Naive Bayes

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import confusion_matrix, accuracy_score

# Dividir o DataFrame em dados de treinamento e teste
X = dados['texto']
y = dados['sentimento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Inicializar os classificadores
mnb = MultinomialNB()
gnb = GaussianNB()
bnb = BernoulliNB()

# Treinar os classificadores
mnb.fit(X_train, y_train)
gnb.fit(X_train.toarray(), y_train)
bnb.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_mnb = mnb.predict(X_test)
y_pred_gnb = gnb.predict(X_test.toarray())
y_pred_bnb = bnb.predict(X_test)



In [13]:
# Gerar as matrizes de confusão
confusion_mtx_mnb = confusion_matrix(y_test, y_pred_mnb)
confusion_mtx_gnb = confusion_matrix(y_test, y_pred_gnb)
confusion_mtx_bnb = confusion_matrix(y_test, y_pred_bnb)

print("Matriz de Confusão - MultinomialNB:")
print(confusion_mtx_mnb)
print("Matriz de Confusão - GaussianNB:")
print(confusion_mtx_gnb)
print("Matriz de Confusão - BernoulliNB:")
print(confusion_mtx_bnb)

Matriz de Confusão - MultinomialNB:
[[26  5]
 [ 8  4]]
Matriz de Confusão - GaussianNB:
[[31  0]
 [ 8  4]]
Matriz de Confusão - BernoulliNB:
[[29  2]
 [ 8  4]]


In [14]:
# Calcular a acurácia dos modelos
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
accuracy_bnb = accuracy_score(y_test, y_pred_bnb)

print("Acurácia - MultinomialNB:", accuracy_mnb)
print("Acurácia - GaussianNB:", accuracy_gnb)
print("Acurácia - BernoulliNB:", accuracy_bnb)

Acurácia - MultinomialNB: 0.6976744186046512
Acurácia - GaussianNB: 0.813953488372093
Acurácia - BernoulliNB: 0.7674418604651163


Suporte Vector Machine

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

# Dividir o DataFrame em dados de treinamento e teste
X = dados['texto']
y = dados['sentimento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Inicializar o classificador SVM
svm = SVC()

# Treinar o classificador SVM
svm.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_svm = svm.predict(X_test)

# Gerar a matriz de confusão
confusion_mtx_svm = confusion_matrix(y_test, y_pred_svm)

print("Matriz de Confusão - SVM:")
print(confusion_mtx_svm)

# Calcular a acurácia do modelo SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print("Acurácia - SVM:", accuracy_svm)


Matriz de Confusão - SVM:
[[31  0]
 [12  0]]
Acurácia - SVM: 0.7209302325581395


Redes Neurais

A rede neural utilizada no exemplo é um Perceptron de Múltiplas Camadas (Multilayer Perceptron, MLP). O MLP é uma arquitetura de rede neural artificial que consiste em várias camadas de neurônios, incluindo uma camada de entrada, uma ou mais camadas ocultas e uma camada de saída. Cada neurônio em uma camada está conectado a todos os neurônios na camada seguinte, formando uma rede densamente conectada.

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

# Dividir o DataFrame em dados de treinamento e teste
X = dados['texto']
y = dados['sentimento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Inicializar o classificador MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), random_state=42)

# Treinar o classificador MLP
mlp.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_mlp = mlp.predict(X_test)

# Gerar a matriz de confusão
confusion_mtx_mlp = confusion_matrix(y_test, y_pred_mlp)

print("Matriz de Confusão - MLP:")
print(confusion_mtx_mlp)

# Calcular a acurácia do modelo MLP
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

print("Acurácia - MLP:", accuracy_mlp)


Matriz de Confusão - MLP:
[[29  2]
 [ 4  8]]
Acurácia - MLP: 0.8604651162790697
